# Kowal LLM (JSON edition)

### Loading JSON

In [1]:
import json

json_path = 'Data/kowal.json'
with open(json_path, 'r', encoding='utf-8') as f:
    npc_data = json.load(f)

# Tworzymy tekst z zawartości JSON jako dokument źródłowy
from langchain.docstore.document import Document

def flatten_npc_to_text(data):
    result = []
    result.append(f"{data['imie']} - {data['rola']}")
    result.append(data["opis"])
    result.append(f"Nastawienie do gracza: {data.get('nastawienie_do_gracza', 'neutralne')}")
    result.append("Przedmioty w ofercie:")
    for item in data["przedmioty"]:
        result.append(f"- {item['nazwa']} ({item['cena']})")
    result.append("Relacje:")
    result.append(f"  Lubi: {', '.join(data['relacje'].get('lubi', []))}")
    result.append(f"  Nie lubi: {', '.join(data['relacje'].get('nie_lubi', []))}")
    result.append("Plotki:")
    for p in data["plotki"]:
        result.append(f"- {p}")
    result.append(f"Waluta: {data['waluta']}")
    return "\n".join(result)

text = flatten_npc_to_text(npc_data)
documents = [Document(page_content=text)]

print("Loaded JSON and created document.")


Loaded JSON and created document.


### Splitting into chunks

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_documents(documents)
print(f"Split JSON into {len(chunks)} chunks")


Split JSON into 2 chunks


### Embeddings

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("Using embedding model: sentence-transformers/all-MiniLM-L6-v2")


Using embedding model: sentence-transformers/all-MiniLM-L6-v2


### Vector store

In [4]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embedding_model)
print("Vectorstore created.")


Vectorstore created.


### RAG pipeline

In [5]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=os.environ['OPENAI_API_KEY'],
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


### say() function

In [6]:
def say(text):
    result = qa_chain.invoke(text)

    print("Odpowiedź:")
    print(result["result"])
    print("\nFragmenty źródłowe:")
    for doc in result["source_documents"]:
        print("-", doc.page_content[:100])


### Testing

In [7]:
say("masz jakies przedmioty masz w swojej ofercie?")
say("wiesz cos o magii?")
say("słyszałem o tobie jakies plotki, opowiedz mi o nich")
say("Lubisz czarodziejów?")
say("za jaką walutę sprzedajesz swoje produkty?")
say("zadasz mi jakies pytanie?")
say("nie jestes zbytnio rozmowny")
say("szukam nowego toporu, masz coś do zaproponowania?")
say("są rasy których nie lubisz?")


Odpowiedź:
W mojej ofercie mam następujące przedmioty:

- Topór bitewny - 25 sztuk złota
- Młot kowalski - 10 sztuk srebra

Jeśli jesteś zainteresowany, mogę pomóc w zakupie!

Fragmenty źródłowe:
- - Pancerz skórzany (30 sztuk złota)
Relacje:
  Lubi: żołnierzy, łowców
  Nie lubi: czarodziejów, kup
- Borin - Kowal
Stary kowal z wioski Dębowy Gród, znany z wyrobu broni i narzędzi najwyższej jakości.

Odpowiedź:
Nie wiem.

Fragmenty źródłowe:
- - Pancerz skórzany (30 sztuk złota)
Relacje:
  Lubi: żołnierzy, łowców
  Nie lubi: czarodziejów, kup
- Borin - Kowal
Stary kowal z wioski Dębowy Gród, znany z wyrobu broni i narzędzi najwyższej jakości.

Odpowiedź:
Słyszałeś, że kiedyś walczyłem w wielkiej wojnie krasnoludów. To był czas, kiedy stawialiśmy czoła końcu naszej niezależności. Dodatkowo, posiadam wiedzę na temat sekretnej techniki kucia zaklętych ostrzy, co przydaje się w moim rzemiośle. Co chciałbyś wiedzieć więcej?

Fragmenty źródłowe:
- - Pancerz skórzany (30 sztuk złota)
Relacje:
 